In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tflearn
from keras.utils import to_categorical

%matplotlib inline

In [2]:
input_path = '~/data/biddings.csv'
data = pd.read_csv(input_path)
print(data.shape)

train = data[:800000]
test = data[800000:]

sample = train.sample(frac=1)
features = sample.drop('convert', axis=1).values
labels = to_categorical(sample.convert.ravel())


test_features = test.drop('convert', axis=1).values
test_labels = to_categorical(test.convert.ravel())

(1000000, 89)


In [10]:
input_layer = tflearn.input_data(shape=[None, features.shape[1]])
dense1 = tflearn.fully_connected(input_layer, 64, weights_init=tflearn.initializations.normal(),
                                 regularizer='L2', weight_decay=0.001)
dense2 = tflearn.fully_connected(dense1, 32, weights_init=tflearn.initializations.normal(),
                                 regularizer='L2', weight_decay=0.001)

softmax = tflearn.fully_connected(dense2, 2, activation='softmax')

# Regression using SGD with learning rate decay
sgd = tflearn.SGD(learning_rate=0.01, lr_decay=0.02)
net = tflearn.regression(softmax, optimizer=sgd,
                         loss='roc_auc_score')

# Training
model = tflearn.DNN(net, tensorboard_verbose=2)

/Users/ytu/.local/share/virtualenvs/rtb-bjTeWfqO/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler

print(features.shape, labels.shape)
rus = RandomUnderSampler(ratio={0: 1531*20, 1: 1531})
# ros = RandomOverSampler(ratio={0: 1531*20, 1: 1531*5})
smote = SMOTE(n_jobs=-1, random_state=42)
# smoteenn = SMOTEENN(smote=SMOTE(n_jobs=-1))

print("Resampling")
resampled_features, resampled_labels = rus.fit_sample(features, labels[:,1])
resampled_features, resampled_labels = smote.fit_sample(resampled_features, resampled_labels)

resampled_labels = to_categorical(resampled_labels, 2)

print("Resampling done")

model.fit(resampled_features, resampled_labels, n_epoch=3, batch_size=16, show_metric=True)

(800000, 88) (800000, 2)
Resampling
Resampling done


IndexError: list index out of range

In [8]:
from sklearn.metrics import *

def rtb_confusion_matrix(test_labels, y_preds):
    m = confusion_matrix(test_labels[:,1], y_preds.argmax(axis=-1))

    print("Confusion Matrix:")
    print("True Negative = %d" % m[0][0])
    print("False Negative = %d" % m[1][0])
    print("True Positive = %d" % m[1][1])
    print("False Positive = %d" % m[0][1])
    

def rtb_f1_score(test_labels, y_preds):
    f = f1_score(test_labels[:,1], y_preds.argmax(axis=-1))
    print("f1 score = %0.3f" % f)
    
def rtb_precision_recall(test_labels, y_preds):
    precision, recall, fbeta_score, support = precision_recall_fscore_support(
        test_labels[:,1], y_preds.argmax(axis=-1))
    print("Precision = %0.3f, Recall = %0.3f" % (np.mean(precision), np.mean(recall)))
    return precision, recall

y_preds = model.predict(test_features)
print(y_preds.shape)

(200000, 2)


In [9]:
rtb_confusion_matrix(test_labels, y_preds)
rtb_f1_score(test_labels, y_preds)
rtb_precision_recall(test_labels, y_preds)
print(roc_auc_score(test_labels, y_preds))
print(tflearn.objectives.roc_auc_score(y_preds, test_labels))

Confusion Matrix:
True Negative = 144811
False Negative = 194
True Positive = 183
False Positive = 54812
f1 score = 0.007
Precision = 0.501, Recall = 0.605
0.656747268999943
Tensor("RocAucScore_2/Sum:0", shape=(), dtype=float32)


In [2]:
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)

# Building deep neural network
input_layer = tflearn.input_data(shape=[None, 784])
dense1 = tflearn.fully_connected(input_layer, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout1 = tflearn.dropout(dense1, 0.8)
dense2 = tflearn.fully_connected(dropout1, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout2 = tflearn.dropout(dense2, 0.8)
softmax = tflearn.fully_connected(dropout2, 10, activation='softmax')

# Regression using SGD with learning rate decay and Top-3 accuracy
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=1000)
top_k = tflearn.metrics.Top_k(3)
net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                         loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=20, validation_set=(testX, testY),
          show_metric=True, run_id="dense_model")

Training Step: 2579  | total loss: 0.31696 | time: 2.474s
| SGD | epoch: 003 | loss: 0.31696 - top3: 0.9829 -- iter: 54976/55000


KeyboardInterrupt: 

In [18]:
print(X.shape, Y.shape)

(55000, 784) (55000, 10)
